In [1]:
# Francisco Dominguez Mateos
# 07/09/2021
# Williams Sindrome detection of: disease, gender, identity, age

In [2]:
"""
Automatic speech recognition and speaker identity
"""
from __future__ import division, print_function, absolute_import
import os
import random
import numpy as np
import sklearn.preprocessing as prep
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout
from tensorflow.keras.layers import Embedding, Bidirectional,Conv1D,GlobalMaxPooling1D,LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses
print("tf=",tf.__version__)
print("keras=",keras.__version__)

tf= 2.3.0
keras= 2.4.0


In [3]:
print("Imports doned")
def wav2mfcc(file_path, max_pad_len=30):
    wave, sampleRate = librosa.load(file_path, mono=True, sr=None)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sampleRate).transpose()
    #print(sampleRate,"wave=",wave.shape,"mfcc=",mfcc.shape,"file=",file_path)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc


def shuffle_list(*ls):
  l =list(zip(*ls))
  random.shuffle(l)
  return zip(*l)

DISEASE_NAME="SW"
#PATH_DATA="/home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201"
PATH_DATA_F="/home/spika/datasets/sonido/voz/diseases/sw/asr_sw_f_21090701"
PATH_DATA_M="/home/spika/datasets/sonido/voz/diseases/sw/asr_sw_m_21090701"
# load wav files with this format speaker_word_number.wav
class EasyASR(object):
    def __init__(self):
        '''
        Constructor
        '''
        self.max_pad_len=44
        self.nval=20
        basepathF=PATH_DATA_F
        basepathM=PATH_DATA_M
        self.mfccs_n=[]
        self.speakers_n=[]
        self.diseases_n=[]
        self.ages_n=[]
        self.genders_n=[]
        self.mfccs_d=[]
        self.speakers_d=[]
        self.diseases_d=[]
        self.ages_d=[]
        self.genders_d=[]
        fNamesF=os.listdir(basepathF)
        print("fNamesF=",len(fNamesF))
        random.shuffle(fNamesF)
        for i,fn in enumerate(fNamesF):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker-desease-age-number.wav
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if disease!=DISEASE_NAME and disease!="N": print("wrong disease label=",disease)
            if disease!=DISEASE_NAME: continue
            mfccs=wav2mfcc(basepathF+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs_d.append(mfccs)
            self.speakers_d.append(speaker)
            self.diseases_d.append(disease)
            self.ages_d.append(age)
            self.genders_d.append("F")
        fNamesM=os.listdir(basepathM)
        print("fNamesM=",len(fNamesM))
        random.shuffle(fNamesM)
        for i,fn in enumerate(fNamesM):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker-desease-age-number.wav
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if disease!=DISEASE_NAME and disease!="N": print("wrong disease label=",disease)
            if disease!=DISEASE_NAME: continue
            mfccs=wav2mfcc(basepathM+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs_d.append(mfccs)
            self.speakers_d.append(speaker)
            self.diseases_d.append(disease)
            self.ages_d.append(age)
            self.genders_d.append("M")
        nSMS=len(self.mfccs_d)
        print("n",DISEASE_NAME,"=",nSMS)
        #SHUFFLE
        self.mfccs_d,self.speakers_d,self.diseases_d,self.ages_d,self.genders_d=shuffle_list(self.mfccs_d,self.speakers_d,self.diseases_d,self.ages_d,self.genders_d)
        self.mfccs_d,self.speakers_d,self.diseases_d,self.ages_d,self.genders_d=list(self.mfccs_d),list(self.speakers_d),list(self.diseases_d),list(self.ages_d),list(self.genders_d)

        random.shuffle(fNamesF)
        random.shuffle(fNamesM)
        for i,fn in enumerate(fNamesF):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker-desease-age-number.wav
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if disease!="N": continue
            mfccs=wav2mfcc(basepathF+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs_n.append(mfccs)
            self.speakers_n.append(speaker)
            self.diseases_n.append(disease)
            self.ages_n.append(age)
            self.genders_n.append("F")
            idxSMS=random.randint(self.nval,nSMS)#in order to not repeat validation samples
            self.mfccs_d.append(self.mfccs_d[idxSMS])
            self.speakers_d.append(self.speakers_d[idxSMS])
            self.diseases_d.append(self.diseases_d[idxSMS])
            self.ages_d.append(self.ages_d[idxSMS])
            self.genders_d.append(self.genders_d[idxSMS])
        for i,fn in enumerate(fNamesM):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker-desease-age-number.wav
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if disease!="N": continue
            mfccs=wav2mfcc(basepathM+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs_n.append(mfccs)
            self.speakers_n.append(speaker)
            self.diseases_n.append(disease)
            self.ages_n.append(age)
            self.genders_n.append("M")
            idxSMS=random.randint(self.nval,nSMS)#in order to not repeat validation samples
            self.mfccs_d.append(self.mfccs_d[idxSMS])
            self.speakers_d.append(self.speakers_d[idxSMS])
            self.diseases_d.append(self.diseases_d[idxSMS])
            self.ages_d.append(self.ages_d[idxSMS])
            self.genders_d.append(self.genders_d[idxSMS])
        #SHUFFLE
        self.mfccs_n,self.speakers_n,self.diseases_n,self.ages_n,self.genders_n=shuffle_list(self.mfccs_n,self.speakers_n,self.diseases_n,self.ages_n,self.genders_n)
        self.mfccs_n,self.speakers_n,self.diseases_n,self.ages_n,self.genders_n=list(self.mfccs_n),list(self.speakers_n),list(self.diseases_n),list(self.ages_n),list(self.genders_n)

        self.mfccs_val   =self.mfccs_d   [:self.nval]+self.mfccs_n   [:self.nval]
        self.speakers_val=self.speakers_d[:self.nval]+self.speakers_n[:self.nval]
        self.diseases_val=self.diseases_d[:self.nval]+self.diseases_n[:self.nval]
        self.ages_val    =self.ages_d    [:self.nval]+self.ages_n    [:self.nval]
        self.genders_val =self.genders_d [:self.nval]+self.genders_n [:self.nval]

        self.mfccs   =self.mfccs_d   [nSMS+self.nval:]+self.mfccs_n   [self.nval:]
        self.speakers=self.speakers_d[nSMS+self.nval:]+self.speakers_n[self.nval:]
        self.diseases=self.diseases_d[nSMS+self.nval:]+self.diseases_n[self.nval:]
        self.ages    =self.ages_d    [nSMS+self.nval:]+self.ages_n    [self.nval:]
        self.genders =self.genders_d [nSMS+self.nval:]+self.genders_n [self.nval:]

        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers+self.speakers_val)
        self.iSpeakerLabels    =self.leSpeakers.transform(self.speakers)
        self.iSpeakerLabels_val=self.leSpeakers.transform(self.speakers_val)
        
        self.leDiseases=prep.LabelEncoder()
        self.leDiseases.fit(self.diseases+self.diseases_val)
        self.iDiseaseLabels    =self.leDiseases.transform(self.diseases)
        self.iDiseaseLabels_val=self.leDiseases.transform(self.diseases_val)
        
        self.leAges=prep.LabelEncoder()
        self.leAges.fit(self.ages+self.ages_val)
        self.iAgeLabels    =self.leAges.transform(self.ages)
        self.iAgeLabels_val=self.leAges.transform(self.ages_val)
        
        self.leGenders=prep.LabelEncoder()
        self.leGenders.fit(self.genders+self.genders_val)
        self.iGendersLabels    =self.leGenders.transform(self.genders)
        self.iGendersLabels_val=self.leGenders.transform(self.genders_val)
        
        print("Samples=",len(self.diseases))
        print(DISEASE_NAME,"=",self.diseases.count(DISEASE_NAME))
        print("N   =",self.diseases.count('N'))
        print(set(self.diseases))
        print(set(self.ages))
        print(set(self.speakers))
        print(set(self.genders))
        print(len(self.diseases))
        print(len(self.genders))
        print(len(self.iGendersLabels))
        print(len(self.iGendersLabels_val))

ea=EasyASR()
print(ea.leDiseases)

Imports doned
fNamesF= 762
fNamesM= 924
n SW = 109
Samples= 3114
SW = 1557
N   = 1557
{'N', 'SW'}
{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 21, 23, 27, 31}
{'NVA', 'PMA', 'GMD', 'DPN', 'FOV', 'RRJ', 'HSI', 'VML', '10V', 'LDM', 'PMC', 'JPI', 'QMA', 'SFL', 'IHI', 'DDR', 'XMA', 'CFM', 'AAI', 'ICJ', 'MTA', 'IMJ', 'FLS', 'VPB', 'BBR', 'CGC', 'CCN', 'LFJ', 'MCJ', 'MOA', 'FSM', 'TDM', 'FDN', 'MMC', 'PMP', 'BGE', 'RPA', 'HPP', 'BBE', 'EMA', 'SRL', 'CMN', 'CRO', 'RRS', 'CFR', 'HJS', 'MAP', 'SOS', 'MSE', 'RMA', 'HXA', 'MMI', 'MDS', 'RCC', 'GRA', 'PMV', 'AMA', 'KSY', 'STA', 'RSA', 'RLG', 'ROJ', 'SMA', 'AAE', 'MMN', 'PDI', 'MPA', 'RBB', 'EMP', 'MCA', 'HGR', 'JCL', 'IVM', 'RMM', 'VFH', 'HCL', 'VCJ', 'FEA', 'PDP', 'KHS', 'VTR', 'RMI', 'LMA', 'ASN', 'BCB', 'GPC', 'LAM', 'NVD', 'SIA', 'YMA', 'OFJ', 'TOA', 'API', 'TJI', 'PRA', 'EPS', 'PAJ', 'BCA', 'BDC', 'MCN', 'CAA', 'ORD', 'RCE', 'TVD', 'RMD', 'SBL', 'ACD', 'BFR', 'MRL', '11C', 'UOM', 'ZSD', 'MML', 'TBD', 'MMA', 'FGI', 'TMR', 'PSE', 'ERA', 'OPC', 'HM

In [5]:
trainX, trainYdiseases, trainYspeakers, trainYages, trainYgenders = np.array(ea.mfccs),np.array(ea.iDiseaseLabels),np.array(ea.iSpeakerLabels),np.array(ea.iAgeLabels),np.array(ea.iGendersLabels)
#testX, testY = test
valX,   valYdiseases,   valYspeakers,   valYages,   valYgenders   = np.array(ea.mfccs_val),np.array(ea.iDiseaseLabels_val),np.array(ea.iSpeakerLabels_val),np.array(ea.iAgeLabels_val),np.array(ea.iGendersLabels_val)

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputDiseases_dim=len(ea.leDiseases.classes_)
outputSpeakers_dim=len(ea.leSpeakers.classes_)
outputAges_dim    =len(ea.leAges.classes_)
outputGenders_dim =len(ea.leGenders.classes_)

print(outputDiseases_dim,outputSpeakers_dim,outputAges_dim,outputGenders_dim)

# Converting labels to binary vectors
trainYdiseases_cat = utils.to_categorical(trainYdiseases,outputDiseases_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
trainYages_cat     = utils.to_categorical(trainYages    ,outputAges_dim)
trainYgenders_cat  = utils.to_categorical(trainYgenders ,outputGenders_dim)
print(len(ea.iGendersLabels),3124-2276)
print(ea.iGendersLabels)
print(trainYgenders.shape)
print(trainYgenders_cat.shape)
print (trainX.shape,trainYdiseases_cat.shape,trainYspeakers_cat.shape,trainYages_cat.shape,trainYgenders_cat.shape)

# Converting labels to binary vectors
valYdiseases_cat = utils.to_categorical(valYdiseases,outputDiseases_dim)
valYspeakers_cat = utils.to_categorical(valYspeakers,outputSpeakers_dim)
valYages_cat     = utils.to_categorical(valYages    ,outputAges_dim)
valYgenders_cat  = utils.to_categorical(valYgenders ,outputGenders_dim)
print (valX.shape,valYdiseases_cat.shape,valYspeakers_cat.shape,valYages_cat.shape,valYgenders_cat.shape)

2 335 16 2
3114 848
[1 1 0 ... 1 1 0]
(3114,)
(3114, 2)
(3114, 44, 20) (3114, 2) (3114, 335) (3114, 16) (3114, 2)
(40, 44, 20) (40, 2) (40, 335) (40, 16) (40, 2)


In [6]:
# Network building
def build_model_asr_lstm(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim,outputGenders_dim,hiden_dim=128):
    DROPOUT_RATE=0.6
    input = Input(shape=(ea.max_pad_len,input_dim))
    #x = LSTM(hiden_dim, input_length=maxlen, input_dim=input_dim)(input)
    x = Bidirectional(LSTM(hiden_dim,return_sequences=True))(input)
    x = Dropout(DROPOUT_RATE)(x)
    x = Bidirectional(LSTM(hiden_dim,return_sequences=True))(x)
    x = Dropout(DROPOUT_RATE)(x)
    x = Bidirectional(LSTM(hiden_dim))(x)
    x = Dropout(DROPOUT_RATE)(x)

    # We add a vanilla hidden layer:
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(DROPOUT_RATE)(x)
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(DROPOUT_RATE)(x)
    wd= Dense(hiden_dim, activation="relu")(x)
    sd= Dense(hiden_dim, activation="relu")(x)
    ad= Dense(hiden_dim, activation="relu")(x)
    gd= Dense(hiden_dim, activation="relu")(x)
    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_diseases = Dense(outputDiseases_dim, activation="softmax", name="disease")(wd)
    prediction_speakers = Dense(outputSpeakers_dim, activation="softmax", name="speaker")(sd)
    prediction_ages     = Dense(outputAges_dim    , activation="softmax", name="age"    )(ad)
    prediction_gender   = Dense(outputGenders_dim , activation="softmax", name="gender" )(gd)

    model = Model(input, [prediction_diseases,prediction_speakers,prediction_ages,prediction_gender])
    return model

In [7]:
model=build_model_asr_lstm(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim,outputGenders_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 44, 20)]     0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 44, 256)      152576      input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 44, 256)      0           bidirectional[0][0]              
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 44, 256)      394240      dropout[0][0]                    
_______________________________________________________________________________________

In [8]:
checkpoint_filepath="checkpoints/sms/"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_disease_accuracy',
    mode='max',
    save_best_only=True)

In [9]:
# Training
history=model.fit(trainX, [trainYdiseases_cat,trainYspeakers_cat,trainYages_cat,trainYgenders_cat], batch_size=100,epochs=150,validation_data=(valX, [valYdiseases_cat,valYspeakers_cat,valYages_cat,valYgenders_cat]))

Epoch 1/150
32/32 [==============================] - 12s 380ms/step - loss: 9.6197 - disease_loss: 0.7060 - speaker_loss: 5.5736 - age_loss: 2.6263 - gender_loss: 0.7137 - disease_accuracy: 0.5048 - speaker_accuracy: 0.0234 - age_accuracy: 0.1346 - gender_accuracy: 0.5263 - val_loss: 9.5060 - val_disease_loss: 0.6710 - val_speaker_loss: 5.4960 - val_age_loss: 2.6633 - val_gender_loss: 0.6756 - val_disease_accuracy: 0.5000 - val_speaker_accuracy: 0.0000e+00 - val_age_accuracy: 0.0250 - val_gender_accuracy: 0.6250
Epoch 2/150
32/32 [==============================] - 14s 451ms/step - loss: 8.8249 - disease_loss: 0.6189 - speaker_loss: 5.0291 - age_loss: 2.4771 - gender_loss: 0.6999 - disease_accuracy: 0.6390 - speaker_accuracy: 0.0376 - age_accuracy: 0.1567 - gender_accuracy: 0.5324 - val_loss: 9.1990 - val_disease_loss: 0.5381 - val_speaker_loss: 5.3676 - val_age_loss: 2.6154 - val_gender_loss: 0.6779 - val_disease_accuracy: 0.7500 - val_speaker_accuracy: 0.0000e+00 - val_age_accuracy: 0

In [74]:
idxSample=288
print(trainX[idxSample:idxSample+1,:,:].shape)
predDisease,predSpeaker,predAge,predGender=model.predict(trainX[idxSample:idxSample+1,:,:])
#print(predDisease,predAge)
y_predDisease= np.argmax(predDisease)
y_predSpeaker= np.argmax(predSpeaker)
y_predAge    = np.argmax(predAge)
y_predGender   = np.argmax(predGender)
print(y_predDisease,"=",np.max(predDisease))
print(y_predSpeaker,"=",np.max(predSpeaker))
print(y_predAge,"="    ,np.max(predAge))
print(y_predAge,"="    ,np.max(predGender))
disease=ea.leDiseases.inverse_transform([y_predDisease])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
age    =    ea.leAges.inverse_transform([y_predAge])
gender =    ea.leGenders.inverse_transform([y_predGender])
print("Predictions:")
print(speaker[0]," disease is \'",disease[0],"\' with age=",age[0])
print("Groundtruth:")
print("speaker=",ea.speakers[idxSample])
print("disease=",ea.diseases[idxSample])
print("ages   =",ea.ages[idxSample])
print("gender =",ea.genders[idxSample])
print()
print(ea.leDiseases.classes_)
print(ea.leSpeakers.classes_)
print(ea.leAges.classes_)
print(ea.leGenders.classes_)

(1, 44, 20)
1 = 0.9987111
34 = 0.99999213
3 = 0.9998969
3 = 0.99999726
Predictions:
BCN  disease is ' SW ' with age= 6
Groundtruth:
speaker= BCN
disease= SW
ages   = 6
gender = M

['N' 'SW']
['10C' '10I' '10J' '10L' '10S' '10V' '11C' '11P' 'AAE' 'AAI' 'AAJ' 'ABN'
 'ACA' 'ACD' 'ACI' 'ACR' 'ADD' 'ADS' 'AFE' 'AIE' 'AIO' 'AIP' 'AMA' 'AMC'
 'API' 'ARM' 'ASC' 'ASN' 'AZM' 'BBA' 'BBE' 'BBR' 'BCA' 'BCB' 'BCN' 'BDC'
 'BFA' 'BFR' 'BGE' 'BHH' 'BMM' 'BPC' 'BTS' 'CAA' 'CAR' 'CCC' 'CCM' 'CCN'
 'CFE' 'CFM' 'CFR' 'CGA' 'CGC' 'CGJ' 'CII' 'CLL' 'CLR' 'CMA' 'CMI' 'CMN'
 'CMS' 'CPA' 'CPD' 'CPM' 'CRC' 'CRD' 'CRE' 'CRI' 'CRO' 'CSA' 'CSJ' 'CSR'
 'CTC' 'CVP' 'DAP' 'DDR' 'DFM' 'DHD' 'DLA' 'DMD' 'DMH' 'DPN' 'DRI' 'DRS'
 'EIR' 'ELL' 'EMA' 'EMC' 'EMG' 'EMP' 'EPS' 'ERA' 'ERJ' 'FCD' 'FCL' 'FDN'
 'FDS' 'FEA' 'FFR' 'FGI' 'FGJ' 'FLS' 'FMI' 'FOV' 'FRM' 'FSM' 'FSP' 'GAE'
 'GBA' 'GBD' 'GDA' 'GDT' 'GES' 'GLA' 'GMD' 'GMM' 'GOM' 'GPC' 'GRA' 'GSA'
 'GSP' 'HAM' 'HCL' 'HGC' 'HGH' 'HGR' 'HJS' 'HMF' 'HMJ' 'HPP' 'HSI' 'HXA'
 'ICJ'

In [75]:
idxSample=26
print(valX[idxSample:idxSample+1,:,:].shape)
predDisease,predSpeaker,predAge,predGender=model.predict(valX[idxSample:idxSample+1,:,:])
#print(predDisease,predAge)
y_predDisease= np.argmax(predDisease)
y_predSpeaker= np.argmax(predSpeaker)
y_predAge    = np.argmax(predAge)
y_predGender   = np.argmax(predGender)
print(y_predDisease,"=",np.max(predDisease))
print(y_predSpeaker,"=",np.max(predSpeaker))
print(y_predAge,"="    ,np.max(predAge))
print(y_predAge,"="    ,np.max(predGender))
disease=ea.leDiseases.inverse_transform([y_predDisease])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
age    =    ea.leAges.inverse_transform([y_predAge])
gender =    ea.leGenders.inverse_transform([y_predGender])
print("Predictions:")
print(speaker[0]," disease is \'",disease[0],"\' with age=",age[0])
print("Groundtruth:")
print("speaker=",ea.speakers[idxSample])
print("disease=",ea.diseases[idxSample])
print("ages   =",ea.ages[idxSample])
print("gender =",ea.genders[idxSample])
print()
print(ea.leDiseases.classes_)
print(ea.leSpeakers.classes_)
print(ea.leAges.classes_)
print(ea.leGenders.classes_)

(1, 44, 20)
0 = 0.99851805
78 = 0.030375764
4 = 0.23920295
4 = 0.5921034
Predictions:
DLA  disease is ' N ' with age= 7
Groundtruth:
speaker= BMM
disease= SW
ages   = 23
gender = F

['N' 'SW']
['10C' '10I' '10J' '10L' '10S' '10V' '11C' '11P' 'AAE' 'AAI' 'AAJ' 'ABN'
 'ACA' 'ACD' 'ACI' 'ACR' 'ADD' 'ADS' 'AFE' 'AIE' 'AIO' 'AIP' 'AMA' 'AMC'
 'API' 'ARM' 'ASC' 'ASN' 'AZM' 'BBA' 'BBE' 'BBR' 'BCA' 'BCB' 'BCN' 'BDC'
 'BFA' 'BFR' 'BGE' 'BHH' 'BMM' 'BPC' 'BTS' 'CAA' 'CAR' 'CCC' 'CCM' 'CCN'
 'CFE' 'CFM' 'CFR' 'CGA' 'CGC' 'CGJ' 'CII' 'CLL' 'CLR' 'CMA' 'CMI' 'CMN'
 'CMS' 'CPA' 'CPD' 'CPM' 'CRC' 'CRD' 'CRE' 'CRI' 'CRO' 'CSA' 'CSJ' 'CSR'
 'CTC' 'CVP' 'DAP' 'DDR' 'DFM' 'DHD' 'DLA' 'DMD' 'DMH' 'DPN' 'DRI' 'DRS'
 'EIR' 'ELL' 'EMA' 'EMC' 'EMG' 'EMP' 'EPS' 'ERA' 'ERJ' 'FCD' 'FCL' 'FDN'
 'FDS' 'FEA' 'FFR' 'FGI' 'FGJ' 'FLS' 'FMI' 'FOV' 'FRM' 'FSM' 'FSP' 'GAE'
 'GBA' 'GBD' 'GDA' 'GDT' 'GES' 'GLA' 'GMD' 'GMM' 'GOM' 'GPC' 'GRA' 'GSA'
 'GSP' 'HAM' 'HCL' 'HGC' 'HGH' 'HGR' 'HJS' 'HMF' 'HMJ' 'HPP' 'HSI' 'HXA'
 'IC